In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  StandardScaler
from sklearn import preprocessing
from collections import Counter
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [2]:
#之前对附件1的处理直接copy过来
df1 = pd.read_csv('附件1：估价训练数据.csv',engine='python',header=None,dtype=object)

col = ['carid','tradeTime','brand','serial','model',
        'mileage','color','cityId','carCode','transferCount',
        'seatings','registerDate','licenseDate','country',
        'maketype','modelyear','displacement','gearbox',
        'oiltype','newprice','feature1','feature2','feature3',
        'feature4','feature5','feature6','feature7','feature8',
        'feature9','feature10','feature11','feature12',
        'feature13','feature14','feature15','price']
df1.columns = col


#类型修改
df1['mileage'] = df1['mileage'].astype('float64')
df1['transferCount'] = df1['transferCount'].astype('float64')
df1['seatings'] = df1['seatings'].astype('float64')
df1['displacement'] = df1['displacement'].astype('float64')
df1['newprice'] = df1['newprice'].astype('float64')
df1['price'] = df1['price'].astype('float64')

# 拆1：feature11
df1.loc[df1['feature11']=='1+2', 'feature11'] = '1'
df1.loc[df1['feature11']=='1', 'feature11'] = '2'
df1.loc[df1['feature11']=='3+2', 'feature11'] = '3'
df1.loc[df1['feature11']=='1+2,4+2', 'feature11'] = '4'
df1.loc[df1['feature11']=='5', 'feature11'] = '5'
df1.loc[df1['feature11']=='1,3+2', 'feature11'] = '6'

df1['feature11'] = df1['feature11'].astype(object)


# 拆2：feature12
str_slice = pd.Series(df1["feature12"]) 
info = [s for s in str_slice.str.split()]
length = []
width = []
height = []
volume = []

for i in info:
    sub_info = i[0].split('*')
    length.append(int(sub_info[0]))
    width.append(int(sub_info[1]))
    height.append(int(sub_info[2]))
    volume.append(int(sub_info[0])*int(sub_info[1])*int(sub_info[2]))
    
df1.insert(df1.shape[1], 'feature12-length', length)
df1.insert(df1.shape[1], 'feature12-width', width)
df1.insert(df1.shape[1], 'feature12-height', height)
df1.insert(df1.shape[1], 'feature12-volume', volume)

df1['feature12-length'] = df1['feature12-length'].astype('float64')
df1['feature12-width'] = df1['feature12-width'].astype('float64')
df1['feature12-height'] = df1['feature12-height'].astype('float64')
df1['feature12-volume'] = df1['feature12-volume'].astype('float64')

# 删除原列
df1=df1.drop(df1[["feature12"]], axis=1)


#将字符串转为日期类型,分别记为变量tradeTime、
tradeTime = pd.to_datetime(df1['tradeTime'])
registerDate = pd.to_datetime(df1['registerDate'])
licenseDate = pd.to_datetime(df1['licenseDate'])

#计算registerDate-tradeTime，licenseDate-tradeTime
delta_reg = tradeTime-registerDate
delta_lic = tradeTime-licenseDate

#以月为单位表示时间差
delta_weeks_reg = []
for delta in delta_reg:
    if delta.days % 7 <= 3:
        weeks = delta.days//7
    else:
        weeks = delta.days//7 + 1
    delta_weeks_reg.append(weeks)
delta_weeks_lic = []
for delta in delta_lic:
    if delta.days % 7 <= 3:
        weeks = delta.days//7
    else:
        weeks = delta.days//7 + 1
    delta_weeks_lic.append(weeks)
    

#转为datetime类型
feature13 = pd.to_datetime(df1['feature13'],format='%Y%m')
modelyear = pd.to_datetime(df1['modelyear'])

#计算traedeTime-modelyear，以年为单位
trade_model = []
for i in range(modelyear.shape[0]):
    trade_model.append(tradeTime[i].year-modelyear[i].year)
    
#计算tradeTime-feature13,以月为单位
trade_f13 = []
for i in range(feature13.shape[0]):
    trade_f13.append(tradeTime[i].year*12+tradeTime[i].month-feature13[i].year*12-feature13[i].month)
    

# #加入
df1.insert(df1.shape[1],'tradeTime-registerDate',delta_weeks_reg)
df1.insert(df1.shape[1],'tradeTime-licenseDate',delta_weeks_lic)
df1.insert(df1.shape[1],'tradeTime-modelyear',trade_model)
df1.insert(df1.shape[1],'tradeTime-feature13',trade_f13)
#删除原列
df1 = df1.drop(df1[["tradeTime","registerDate","licenseDate","feature13","modelyear"]],axis=1)

# 修改数据类型
df1["tradeTime-registerDate"] = df1["tradeTime-registerDate"].astype('float64')
df1["tradeTime-licenseDate"] = df1["tradeTime-licenseDate"].astype('float64')

df1.head(3)

carid brand serial model  mileage color cityId carCode  transferCount  \
0     1     1      1     1     4.01     1      1       1            0.0   
1     2     2      2     2     8.60     1      2       1            0.0   
2     5     5      5     5    15.56     1      2       3            0.0   

   seatings         ...         feature15 price  feature12-length  \
0       5.0         ...               NaN  4.24            4220.0   
1       5.0         ...               NaN  7.38            4630.0   
2       5.0         ...               NaN  1.00            4515.0   

  feature12-width feature12-height  feature12-volume tradeTime-registerDate  \
0          1740.0           1625.0      1.193205e+10                  186.0   
1          1775.0           1480.0      1.216301e+10                  238.0   
2          1725.0           1445.0      1.125420e+10                  698.0   

  tradeTime-licenseDate tradeTime-modelyear tradeTime-feature13  
0                 178.0                 4.0                45.0  
1                 222.0                 4.0                57.0  
2                 694.0                13.0                 NaN  

[3 rows x 38 columns]

In [3]:
#第23、26和34列缺失值超过50%，无法填补缺失值，删去这两列
df1=df1.drop(df1[["feature4","feature7","feature15"]], axis=1)

In [4]:
#对df1划分训练集和测试集

# 删除gearbox缺失数据：只有一条
df1 = df1.dropna(subset=["gearbox"],axis=0)

#price插到最后
price = df1['price']
df1.drop(labels=['price'], axis=1,inplace=True)
df1.insert(34, 'price', price)

#划分训练集和测试集
from sklearn.model_selection import  train_test_split

lst = [i for i in range(0,34)]

X=df1.iloc[:,lst]
y=df1.loc[:,"price"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)
print("y_train.shape:",y_train.shape)
print("y_test.shape:",y_test.shape)

X_train.shape: (20999, 34)
X_test.shape: (9000, 34)
y_train.shape: (20999,)
y_test.shape: (9000,)


In [6]:
# 缺失变量：feature11在null_lst1中会报错
null_lst1 = ['carCode','country', 'maketype', 'feature1', 'feature8', 'feature9', 'feature10','feature11']
null_lst2 = [ 'tradeTime-modelyear','tradeTime-feature13']


# 用随机森林分类树填充缺失值

from sklearn.ensemble import RandomForestClassifier
def set_missing_classier_train(df, feature):
    df = df[['brand','serial','model','gearbox',
        'mileage','color','cityId','transferCount',
        'seatings','displacement','oiltype','newprice',
        'feature2','feature3', 'feature5','feature6', 
        'feature14','tradeTime-registerDate','tradeTime-licenseDate',
        'feature12-length','feature12-width','feature12-height','feature12-volume', feature]]
    
    known = df[df[feature].notnull()].values
    unknown = df[df[feature].isnull()].values
    
    y = known[:,-1]
    X = known[:,:-1]
    
    rfc = RandomForestClassifier(random_state=0,n_estimators=100,n_jobs=-1)
    
    rfc.fit(X,y)
    
    predict = rfc.predict(unknown[:,:-1])
    
    df.loc[(df[feature].isnull()),feature] = predict
    
    #return rfr是为后续预测训练集feature缺失数据
    return df,rfc

from sklearn.ensemble import RandomForestClassifier
def set_missing_classier_test(rfc, df, feature):
    df = df[['brand','serial','model','gearbox',
        'mileage','color','cityId','transferCount',
        'seatings','displacement','oiltype','newprice',
        'feature2','feature3', 'feature5','feature6', 
        'feature14','tradeTime-registerDate','tradeTime-licenseDate',
        'feature12-length','feature12-width','feature12-height','feature12-volume', feature]]
    
    known = df[df[feature].notnull()].values
    unknown = df[df[feature].isnull()].values
    
    y = known[:,-1]
    X = known[:,:-1]

    rfc.fit(X,y)
    
    predict = rfc.predict(unknown[:,:-1])
    
    df.loc[(df[feature].isnull()),feature] = predict
    
    #return rfr是为后续预测训练集feature缺失数据
    return df

for i in null_lst1:
    train_result, rfc = set_missing_classier_train(X_train, i)
    test_result = set_missing_classier_test(rfc, X_test, i)
    X_train[i] = train_result[i]
    X_test[i] = test_result[i]
    print(i+"已完成！")
    
# 用随机森林回归树填充缺失值

from sklearn.ensemble import RandomForestRegressor
def set_missing_regress_train(df, feature):
    df = df[['brand','serial','model','gearbox',
        'mileage','color','cityId','transferCount',
        'seatings','displacement','oiltype','newprice',
        'feature2','feature3', 'feature5','feature6', 
        'feature14','tradeTime-registerDate','tradeTime-licenseDate',
        'feature12-length','feature12-width','feature12-height','feature12-volume', feature]]
    
    known = df[df[feature].notnull()].values
    unknown = df[df[feature].isnull()].values
    
    y = known[:,-1]
    X = known[:,:-1]

    rfr = RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
    
    rfr.fit(X,y)
    
    predict = rfr.predict(unknown[:,:-1])
    
    df.loc[(df[feature].isnull()),feature] = predict
    
    #return rfr是为后续预测训练集feature缺失数据
    return df,rfr

# 用随机森林回归树填充缺失值

from sklearn.ensemble import RandomForestRegressor
def set_missing_regress_test(rfr, df, feature):
    df = df[['brand','serial','model','gearbox',
        'mileage','color','cityId','transferCount',
        'seatings','displacement','oiltype','newprice',
        'feature2','feature3', 'feature5','feature6', 
        'feature14','tradeTime-registerDate','tradeTime-licenseDate',
        'feature12-length','feature12-width','feature12-height','feature12-volume', feature]]
    
    known = df[df[feature].notnull()].values
    unknown = df[df[feature].isnull()].values
    
    y = known[:,-1]
    X = known[:,:-1]

    rfr.fit(X,y)
    
    predict = rfr.predict(unknown[:,:-1])
    
    df.loc[(df[feature].isnull()),feature] = predict
    
    #return rfr是为后续预测训练集feature缺失数据
    return df

for i in null_lst2:
    train_result, rfr = set_missing_regress_train(X_train, i)
    test_result = set_missing_regress_test(rfr, X_test, i)
    X_train[i] = train_result[i]
    X_test[i] = test_result[i]
    print(i+"已完成！")

carCode已完成！
country已完成！
maketype已完成！
feature1已完成！
feature8已完成！
feature9已完成！
feature10已完成！
feature11已完成！
tradeTime-modelyear已完成！
tradeTime-feature13已完成！


In [8]:
df1 = pd.concat([X_train,X_test],axis=0)

In [9]:
pd.set_option('display.max_columns', None)

df4 = pd.read_csv('附件4：门店交易训练数据.csv',engine='python',header=None)
df4.columns = ["carid","pushDate","pushPrice","updatePriceTimeJson","pullDate","withdrawDate"]

df4['carid'] = df4['carid'].astype('str')

#转化成时间
df4['pushDate'] = pd.to_datetime(df4['pushDate'])
df4['pullDate'] = pd.to_datetime(df4['pullDate'])
df4['withdrawDate'] = pd.to_datetime(df4['withdrawDate'])
df4.head(3)

carid   pushDate  pushPrice   updatePriceTimeJson   pullDate withdrawDate
0  68603 2021-03-11       3.98                    {} 2021-03-11   2021-03-11
1  12312 2021-05-14       4.50                    {} 2021-06-14   2021-06-14
2  57655 2021-03-13      23.90  {"2021-04-05": "23"} 2021-04-08   2021-04-08

In [10]:
#成交了的车，下架时间=成交时间
#没成交的车，下架时间有，成交时间无
#成交周期=上架时间-成交时间，所以没成交的车的成交时间填0也不合适
df4_success=df4[~pd.isna(df4["withdrawDate"])]
df4_unsuccess=df4[pd.isna(df4["withdrawDate"])]


In [11]:
df=pd.merge(df4,df1,on="carid" ,how="left")
df = df.dropna(subset=["brand"],axis=0)
df

carid   pushDate  pushPrice  \
0     68603 2021-03-11     3.9800   
1     12312 2021-05-14     4.5000   
2     57655 2021-03-13    23.9000   
3     45688 2020-09-01    20.5798   
4     52081 2021-04-29    12.2800   
5      6729 2020-12-02    18.5000   
6       766 2021-07-25     9.8000   
7     10453 2021-01-29    10.8000   
8     65800 2021-04-01    12.2800   
9     36776 2020-07-16    16.8000   
10    30211 2020-11-01    19.3800   
11      624 2021-07-20    11.4200   
12    12588 2020-05-09     9.7000   
13    11435 2020-02-15     8.6800   
14      295 2021-07-10     6.0000   
15    33761 2020-06-30    11.5000   
16      622 2021-07-02     5.8000   
17    35492 2020-12-17    24.8000   
18    69974 2021-04-12     4.5000   
19    54141 2020-06-09    12.0000   
20     4343 2020-09-09     4.6000   
21    71250 2021-07-17    17.5000   
22    18269 2020-09-14     8.6800   
23    14920 2021-01-03    39.6000   
24    60829 2021-04-10     6.8500   
25    18396 2020-08-08    15.8000   
26    16035 2020-03-05     5.5000   
27    68448 2020-06-13    28.8000   
28    10877 2020-08-08     7.3000   
29     1444 2021-04-12     1.9500   
...     ...        ...        ...   
9970  64669 2020-09-18    16.8000   
9971  33676 2020-04-24    23.8000   
9972  64901 2020-11-04    22.6800   
9973  67769 2021-03-02    10.6800   
9974  58096 2020-09-20     4.9000   
9975  61140 2021-03-17    11.9800   
9976  45904 2020-07-31    34.8000   
9977  18196 2021-07-05     3.2800   
9978  14252 2020-11-22    16.7000   
9979  22274 2020-06-17     1.8800   
9980   8752 2020-11-19     7.9800   
9981  31974 2021-04-07     4.0000   
9982  37239 2021-04-15    11.6800   
9983  46121 2020-05-25     2.8000   
9984  53615 2020-09-13    24.8000   
9985  19475 2021-03-13     6.4000   
9986  60454 2020-09-30    17.8000   
9987  27120 2020-12-25    22.5800   
9988   7553 2020-11-19     6.8000   
9989  11007 2021-03-24     9.6800   
9990  63059 2020-08-05     2.0000   
9991  63469 2021-07-17    11.8000   
9992  45307 2021-01-23     9.9000   
9993  57147 2021-04-20    11.3800   
9994  17286 2020-09-13    16.8000   
9995  10177 2021-07-31    22.8800   
9996  54312 2021-03-22    12.3000   
9997  45597 2021-01-09    15.3800   
9998  38629 2021-06-07    36.8800   
9999  58619 2021-03-06    39.3800   

                                    updatePriceTimeJson   pullDate  \
0                                                    {} 2021-03-11   
1                                                    {} 2021-06-14   
2                                  {"2021-04-05": "23"} 2021-04-08   
3                                                    {} 2020-09-04   
4                                {"2021-05-20": "11.9"} 2021-06-21   
5     {"2020-12-29": "17.68", "2020-12-30": "17.8799... 2021-01-20   
6                                                    {} 2021-08-19   
7                                                    {} 2021-03-22   
8                                                    {} 2021-04-04   
9                                                    {} 2020-07-25   
10       {"2020-11-05": "18.98", "2020-11-09": "18.58"} 2020-11-15   
11                                                   {} 2021-07-26   
12                                                   {} 2020-05-23   
13                                {"2020-03-01": "8.6"} 2020-06-11   
14                                                   {} 2021-07-23   
15           {"2020-07-05": "11.3", "2020-07-10": "11"} 2020-07-15   
16                                                   {} 2021-07-04   
17                                                   {} 2020-12-25   
18                               {"2021-04-16": "3.98"} 2021-04-20   
19                                                   {} 2020-06-11   
20                                                   {} 2020-09-09   
21                                                   {} 2021-07-17   
22                                                   {} 2020-10-25   
23                             

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9999
Data columns (total 39 columns):
carid                     9999 non-null object
pushDate                  9999 non-null datetime64[ns]
pushPrice                 9999 non-null float64
updatePriceTimeJson       9999 non-null object
pullDate                  9999 non-null datetime64[ns]
withdrawDate              7999 non-null datetime64[ns]
brand                     9999 non-null object
serial                    9999 non-null object
model                     9999 non-null object
mileage                   9999 non-null float64
color                     9999 non-null object
cityId                    9999 non-null object
carCode                   9999 non-null object
transferCount             9999 non-null float64
seatings                  9999 non-null float64
country                   9999 non-null object
maketype                  9999 non-null object
displacement              9999 non-null float64
gearbox           

In [13]:
outputpath = 'E:/Fanny_Python_Files/MergeResult.csv'
(df).to_csv(outputpath,sep=',',index=False,header=True)